# D3 in jupyter Notebook

Coded forked from https://github.com/skariel/IPython_d3_js_demo

## Dependencies

In [1]:
import pandas as pd
import numpy as np
import time
from IPython import display
from IPython.display import clear_output

import jinja2  as jj


## Jinja Template

We start be creating a jinja template (http://jinja.pocoo.org/)

This will give us an easy way to modify specific sections of our html.

In [25]:
%%writefile jinja_templates/basic.jinja
<!DOCTYPE html>
<meta charset="utf-8">
<style>

body {
  font: 10px sans-serif;
}

.axis path,
.axis line {
  fill: none;
  stroke: #000;
  shape-rendering: crispEdges;
}

.dot {
  stroke: #000;
}

</style>
<body>
<script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min.js"></script>
<input type="button" onclick="clearInterval(timer)" value="stop">
<script>

var margin = {top: 20, right: 20, bottom: 30, left: 40},
    width = {{ width }} - margin.left - margin.right,
    height = {{ height }} - margin.top - margin.bottom;

var x = d3.scale.linear()
    .range([0, width]);

var y = d3.scale.linear()
    .range([height, 0]);

var color = d3.scale.category10();

var svg = d3.select("body").append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");

d3.csv( {{ fn }}, function(error, data) {
  if (error) throw error;

  data.forEach(function(d) {
    d.x = +d.x;
    d.y = +d.y;
  });

  x.domain([0, 10]).nice();
  y.domain([0, 10]).nice();

  svg.selectAll(".dot")
      .data(data)
    .enter().append("circle")
      .attr("class", "dot")
      .attr("r", 3.5)
      .attr("cx", function(d) { return x(d.x); })
      .attr("cy", function(d) { return y(d.y); });

});


var timer = setInterval(function() {
            updateData();
        }, 5000);


function updateData() {
    d3.csv( {{ fn }}, function(error, data) {
      if (error) throw error;
      
      data.forEach(function(d) {
        d.x = +d.x;
        d.y = +d.y;
      });
      
      var circle = svg.selectAll(".dot")
            .data(data, function(d) { return d.ueid; });
        
      circle.transition()
            .duration(1000)
            .attr("cx", function(d) { return x(d.x); })
            .attr("cy", function(d) { return y(d.y); })
            .attr("r", 3.5)
            .attr("fill", "black")
      
      circle.enter().append("circle")
            .attr("class", "dot")
            .attr("r", 3.5)
            .attr("cx", function(d) { return x(d.x); })
            .attr("cy", function(d) { return y(d.y); })
            .attr("fill", "red")
      
      circle.exit().remove()
            
      });
}

</script>

Overwriting jinja_templates/basic.jinja


## Render Template

In [35]:
def render_ue(fn="output.csv", width=600, height=600):
    # Load the template
    templateLoader = jj.FileSystemLoader( searchpath="jinja_templates/")
    templateEnv = jj.Environment( loader=templateLoader )

    TEMPLATE_FILE = "/basic.jinja"

    template = templateEnv.get_template( TEMPLATE_FILE )
    
    # Configure variables
    templateVars = { "width": width,
                     "height": height,
                     "fn": '"' + fn + '"'
                   }

    outputHTML = template.render( templateVars )
    
    # wrap the html in an iframe string
    txt = ("<iframe " +
       "srcdoc='" + outputHTML + "' " +
       "width=" + str(width + 30)+ " height=" + str(height + 30) + ">" +
       "</iframe>")

    # return display object
    return display.HTML(txt) 

In [36]:
render_ue()

Update positions of circles

In [32]:
pos = np.random.rand(10,2) * 10
ueid = range(10)
n = 200
start = time.clock()
duration = 20 #seconds

while time.clock() - start < duration:
    pool = int(1.1 * n)
    
    pos = np.random.rand(pool, 2) * 10
    ueid = range(pool)
    df = pd.DataFrame({'ueid': ueid,
                       'x': pos[:,0],
                       'y': pos[:,1]})
    f = np.random.binomial(1, 1. * n / pool, df.shape[0]).astype('bool')
    df.loc[f, :].to_csv('output.csv')
    clear_output()
    display.display(df.loc[f, :].head(10))
    time.sleep(5)

,ueid,x,y
0,0,7.764671,3.763226
1,1,1.584562,5.303441
2,2,8.821019,0.390577
3,3,9.798424,6.189083
4,4,3.135599,7.398747
5,5,1.835748,8.063930
6,6,3.618552,1.242678
7,7,3.565563,5.688656
8,8,2.036621,0.641884
9,9,3.983812,8.978520
